# A Story of Failures in the Optimisation of Water Distribution Systems

**Author**: Dennis Zanutto

**Supervision**: Andrea Castelletti, Dragan Savić

## About This Notebook

Aware of the computational needs of our idea (joint optimisation of WDS design and operation, which has an exponentially larger search space than just the two individual spaces), I started the development of a C++ library to work as an interface between the pagmo optimisation library framework, which provides great parallel computing capabilities, and the EPANET hydraulic simulation library. However, what I was not expecting was the number of problems and difficulties that I would encounter.

This document explains my journey in this development and is meant to clarify the changes between versions, the errors/bugs found, and how they were solved.

The analysis focuses on the **Anytown network problem** – a well-established benchmark in water distribution system optimization. You can read more about this benchmark [here](https://www.exeter.ac.uk/research/centres/cws/resources/benchmarks/). But at the beginning I tested the **Hanoi problem**. 

The final goal is to have a generic library that can create problems starting only from configuration files (and eventually have benchmark problems hard-coded for convenience).

**Experimental settings**: Parameters and configurations can be found in two places:
- Hard-coded parameters in the source code
- Experiment metadata in the results JSON files (also loaded as dictionaries in this notebook)

## Setup and Running Instructions
Follow the instructions in the GitHub Pages about [building the project](../docs/building.md).

**Note:** You will need to build all the releases, as this notebook goes across the whole development history and shows the breaking changes between versions.

In [ ]:
# Import required libraries
import os
import sys
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as subp
import pandas as pd
import numpy as np

import pygmo as pg

sys.path.append('..')
import pybeme

Now that the libraries are imported, we can use pybeme to load the experimental results from this folder. Each experiment is numbered, and you can inspect the `experiment.data` dictionary to understand its properties and settings.

However, at the beginning (i.e., until v24.04.01-wdsa-ccwi-2024) I was naming them, as names got out of end I stopped.

Moreover, with 24.06.01 I also changed the structure of the configuration and output files. So that the old experiments have been "modernized" and the rename with an absolute numbering. 

> Have a look at `(root)/pybeme/utility/modernize_pre_v240601_exp.py` to understand the changes.

In [ ]:
# Load the experiments, prepare the plots
experiments = pybeme.load_experiments(os.path.join('data', 'all'), verbose=False)

### Plotting Configuration and Helper Functions

The following section sets up consistent styling for all figures and defines helper functions for data processing and visualization.

In [ ]:
# Plot the pareto front of the first round of experiments.
# There is something wrong, so plot the Head and pressure for the tanks, junctions
msize = 10
xlims = [-.5e6, 20.5e6]
ylims = [-0.05, 0.605]
xaxis_title_text='Cost [$]'
yaxis_title_text='OF Reliability Index f1 [-]'
title_font_size=20
legend_font_size=16
def_font_size=12
def fix_layout(a_fig: go.Figure, a_title: str) -> None:
    a_fig.update_layout(title=dict(
                            text=a_title,
                            xanchor='center',
                            x=0.5,
                            yanchor='top',
                            y=1,
                            font_size=title_font_size
                        ),
                        plot_bgcolor='white',
                        paper_bgcolor='white',
                        xaxis=dict(
                            title=xaxis_title_text,
                            range=xlims,
                            automargin=True,
                            showline=True,
                            showgrid=True,
                            linewidth=1,
                            linecolor='grey',
                            zerolinecolor='black',
                            gridcolor='lightgrey'
                        ),
                        yaxis=dict(
                            title=yaxis_title_text,
                            range=ylims,
                            automargin=True,
                            showline=True,
                            showgrid=True,
                            linewidth=1,
                            linecolor='grey',
                            zerolinecolor='black',
                            gridcolor='lightgrey'
                        ),
                        width=1200,
                        height=800,
                        font=dict(
                            family="Lato",
                            color="black",
                            size=def_font_size
                        ),
                        margin=dict(
                            l=10,
                            r=10,
                            b=10,
                            t=50,
                            pad=0
                        ),
                        showlegend=True,
                        legend=dict(
                            orientation="v",
                            xanchor="left",
                            x=0.03,  
                            yanchor="top",
                            y=0.9,  
                            itemsizing='trace',  # To ensure items in legend keep the same size
                            traceorder="normal",
                            bgcolor="White",  # Background color
                            bordercolor="Black",  # Border color
                            borderwidth=1,  # Border width
                            groupclick="toggleitem",
                            itemclick="toggleothers",
                            itemdoubleclick="toggle",
                            tracegroupgap=100,
                            font_size=legend_font_size
                        )
)

def filter_pf(experiment: pybeme.Experiment):
    final_fvs = experiment.final_fitness_vectors.to_numpy()
    pf = pg.non_dominated_front_2d(final_fvs)
    pf = pf[final_fvs[pf,0] < 20e6]
    pf = pf[final_fvs[pf,1] < 0]
    return pf

def plot_pareto_fronts(experiments: dict, exps2plot: list, colors: list, markers: list, names: list):
    fig = go.Figure()

    for e, expname, in enumerate(exps2plot):
        final_fvs = experiments[expname].final_fitness_vectors.to_numpy()
        pf = filter_pf(experiments[expname])
        npf = np.setdiff1d(np.arange(final_fvs.shape[0]), pf)
        
        fig.add_trace(go.Scatter(x=final_fvs[pf,0], y=-final_fvs[pf,1], mode='markers',
                                marker=dict(size=msize, symbol=markers[e], color=colors[e]),  
                                showlegend=True, name=names[e],
                                legendgroup=expname,
                                customdata=np.array(pf, dtype=str),
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                                ) )
        fig.add_trace(go.Scatter(x=final_fvs[npf,0], y=-final_fvs[npf,1], mode='markers', opacity=0.5,
                                marker=dict(size=msize, symbol=markers[e]+'-open', color=colors[e]),  
                                showlegend=True, name=names[e],
                                legendgroup=expname,
                                customdata=np.array(npf, dtype=str),
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                                ) )

    fix_layout(fig, 'Pareto fronts of the different solutions')

    fig.show()

def percentage_errors(pre, post):
    pe = (pre - post) / np.abs(pre)
    # Failed solution have rel indx >= 0
    pe[post[:,1] >= 0, :] = -0.99
    return pe

def version_title_matrix(vers2test: list) -> list:
    assert len(vers2test) == 2, 'Only two versions are supported'
    titles = ["", f"{vers2test[0]} -> {vers2test[1]}", "", f"{vers2test[1]} -> {vers2test[0]}", "", "", "", ""]
    return titles

def plot_version_mismatch():
    fig = subp.make_subplots(rows=2, cols=4, subplot_titles=version_title_matrix(versions2test),
                            shared_xaxes=True, shared_yaxes=True,
                            vertical_spacing=0.01, horizontal_spacing=0.01,
                            row_heights=[0.8, 0.2], column_widths=[0.1, 0.4, 0.1, 0.4],
                            x_title="Evaluation version")
    fig.update_layout(width=1200, height=600)
    for c in range(1,5):
        for r in range(1, 3):
            fig.update_xaxes(showline=True, showgrid=True, linewidth=1, linecolor='grey', zerolinecolor='black', gridcolor='lightgrey', mirror=True,
                             range=[-1.01, 1.01],
                             row=r, col=c)
            fig.update_yaxes(showline=True, showgrid=True, linewidth=1, linecolor='grey', zerolinecolor='black', gridcolor='lightgrey', mirror=True,
                              range=[-1.01, 1.01],
                              row=r, col=c)
            fig.update_layout(plot_bgcolor='white', paper_bgcolor='white') 
    fig.update_xaxes(title_text='Cost percentage error', row=2, col=2)
    fig.update_xaxes(title_text='Cost percentage error', row=2, col=4)
    fig.update_yaxes(title_text='RelIdx percentage error', row=1, col=1)

    for e, expname in enumerate(exps2plot):
        final_fvs = experiments[expname].final_fitness_vectors
        pf_idx = filter_pf(experiments[expname])
        pf_pre = final_fvs.to_numpy()[pf_idx]
        final_indvs_coords = final_fvs.index[pf_idx]

        pf_post = np.zeros_like(pf_pre)
        for i, fic in enumerate(final_indvs_coords):
            final_indv_coord = (fic[0], # island name
                                experiments[expname].generations[fic[0]].to_numpy()[-1], # last generation of the island
                                fic[1]) # individual index

            sim = experiments[expname].simulator(final_indv_coord)
            
            if e % 2 == 0:
                valt = versions2test[1]
                valt_column = 2
            else:
                valt = versions2test[0]
                valt_column = 4
            
            sim.data['bemelib_version'] = valt
            sim.run()
            pf_post[i] = sim.result

        pe = percentage_errors(pf_pre, pf_post)
        pe_sux = pe[pe[:,1] != -0.99, :]
        fig.add_trace(go.Scatter(x=pe_sux[:,0], y=pe_sux[:,1], mode='markers',
                                marker=dict(size=msize, symbol=markers[e], color=colors[e]),
                                showlegend=False,
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f}'
                                ), row=1, col=valt_column)
        pe_fail = pe[pe[:,1] == -0.99, :]
        fig.add_trace(go.Scatter(x=pe_fail[:,0], y=pe_fail[:,1], mode='markers',
                                marker=dict(size=msize, symbol=markers[e], color=colors[e], line=dict(color='red', width=1)),  
                                showlegend=False,
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f}'
                                ), row=1, col=valt_column)
        
        # Add the distribution of the cost error, in the same column, but second row
        deltac_counts, deltac_bins = np.histogram(pe[:,0], bins=np.linspace(-1, 1, 20))
        fig.add_trace(go.Bar(x=deltac_bins+((deltac_bins[1]-deltac_bins[0])/2), y=deltac_counts/np.sum(deltac_counts), width=(deltac_bins[1]-deltac_bins[0]),
                            opacity=0.75,
                            marker=dict(color=colors[e]), showlegend=False), row=2, col=valt_column)

        # Add the distribution of the reliability index error, in the first row, but one column before
        deltar_counts, deltar_bins = np.histogram(pe[:,1], bins=np.linspace(-1, 1, 20))
        fig.add_trace(go.Bar(y=deltar_bins+((deltar_bins[1]-deltar_bins[0])/2), x=deltar_counts/np.sum(deltar_counts), width=(deltar_bins[1]-deltar_bins[0]), orientation='h',
                            opacity=0.75,
                            marker=dict(color=colors[e]), showlegend=False), row=1, col=valt_column-1)

        # Add the centroid of the error distribution
        fig.add_trace(go.Scatter(x=[np.mean(pe[:,0])], y=[np.mean(pe[:,1])], mode='markers',
                                marker=dict(size=2*msize, symbol='x-open', color=colors[e], line=dict(color=colors[e], width=1)),  
                                showlegend=False,
                                customdata=[np.mean(pe[:,0]), np.mean(pe[:,1])],
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                                ), row=1, col=valt_column)
    
    fig.update_layout(barmode='overlay')
    fig.show()

def plot_formulation_mismatch():
    fig = subp.make_subplots(rows=2, cols=4, subplot_titles=version_title_matrix(formulations2test),
                            shared_xaxes=True, shared_yaxes=True,
                            vertical_spacing=0.01, horizontal_spacing=0.01,
                            row_heights=[0.8, 0.2], column_widths=[0.1, 0.4, 0.1, 0.4],
                            x_title="Evaluation Formulation")
    fig.update_layout(width=1200, height=600)
    for c in range(1,5):
        for r in range(1, 3):
            fig.update_xaxes(showline=True, showgrid=True, linewidth=1, linecolor='grey', zerolinecolor='black', gridcolor='lightgrey', mirror=True,
                             range=[-1.01, 1.01],
                             row=r, col=c)
            fig.update_yaxes(showline=True, showgrid=True, linewidth=1, linecolor='grey', zerolinecolor='black', gridcolor='lightgrey', mirror=True,
                              range=[-1.01, 1.01],
                              row=r, col=c)
            fig.update_layout(plot_bgcolor='white', paper_bgcolor='white') 
    fig.update_xaxes(title_text='Cost percentage error', row=2, col=2)
    fig.update_xaxes(title_text='Cost percentage error', row=2, col=4)
    fig.update_yaxes(title_text='RelIdx percentage error', row=1, col=1)

    for e, expname in enumerate(exps2plot):
        final_fvs = experiments[expname].final_fitness_vectors
        pf_idx = filter_pf(experiments[expname])
        pf_pre = final_fvs.to_numpy()[pf_idx]
        final_indvs_coords = final_fvs.index[pf_idx]

        pf_post = np.zeros_like(pf_pre)
        for i, fic in enumerate(final_indvs_coords):
            final_indv_coord = (fic[0], # island name
                                experiments[expname].generations[fic[0]].to_numpy()[-1], # last generation of the island
                                fic[1]) # individual index

            sim = experiments[expname].simulator(final_indv_coord)

            # Convert from one formulation to another:
            if e % 2 == 0:
                sim.convert_to_formulation(formulations2test[1])
                valt_column = 2
            else:
                sim.convert_to_formulation(formulations2test[0])
                valt_column = 4
            
            sim.run()
            pf_post[i] = sim.result

        pe = percentage_errors(pf_pre, pf_post)
        pe_sux = pe[pe[:,1] != -0.99, :]
        fig.add_trace(go.Scatter(x=pe_sux[:,0], y=pe_sux[:,1], mode='markers',
                                marker=dict(size=msize, symbol=markers[e], color=colors[e]),
                                showlegend=False,
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f}'
                                ), row=1, col=valt_column)
        pe_fail = pe[pe[:,1] == -0.99, :]
        fig.add_trace(go.Scatter(x=pe_fail[:,0], y=pe_fail[:,1], mode='markers',
                                marker=dict(size=msize, symbol=markers[e], color=colors[e], line=dict(color='red', width=1)),  
                                showlegend=False,
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f}'
                                ), row=1, col=valt_column)
        
        # Add the distribution of the cost error, in the same column, but second row
        deltac_counts, deltac_bins = np.histogram(pe[:,0], bins=np.linspace(-1, 1, 20))
        fig.add_trace(go.Bar(x=deltac_bins+((deltac_bins[1]-deltac_bins[0])/2), y=deltac_counts/np.sum(deltac_counts), width=(deltac_bins[1]-deltac_bins[0]),
                            opacity=0.75,
                            marker=dict(color=colors[e]), showlegend=False), row=2, col=valt_column)

        # Add the distribution of the reliability index error, in the first row, but one column before
        deltar_counts, deltar_bins = np.histogram(pe[:,1], bins=np.linspace(-1, 1, 20))
        fig.add_trace(go.Bar(y=deltar_bins+((deltar_bins[1]-deltar_bins[0])/2), x=deltar_counts/np.sum(deltar_counts), width=(deltar_bins[1]-deltar_bins[0]), orientation='h',
                            opacity=0.75,
                            marker=dict(color=colors[e]), showlegend=False), row=1, col=valt_column-1)

        # Add the centroid of the error distribution
        fig.add_trace(go.Scatter(x=[np.mean(pe[:,0])], y=[np.mean(pe[:,1])], mode='markers',
                                marker=dict(size=2*msize, symbol='x-open', color=colors[e], line=dict(color=colors[e], width=1)),  
                                showlegend=False,
                                customdata=[np.mean(pe[:,0]), np.mean(pe[:,1])],
                                hovertemplate='Cost: %{x:2.2f} <br> Reliability Index: %{y:.2f} <extra>%{customdata}</extra>'
                                ), row=1, col=valt_column)
    
    fig.update_layout(barmode='overlay')
    fig.show()

## R1: EGU 2024

I produce the results for my presentation at EGU where I tried to compare the integrated approach (both deisgn decision variable and control ones are optimized), the pure design approach (later called design-only - where the operations are fixed a priori - we tried with different pumping patterns), and a integrated-looped where operarions are optimized for each design that is tested. the latter is not present here because I did not convert those experiments as I could not reproduce them anymore (I did not track a parameter for the population used internally).

so, in this plot we look at the resultign pareto front of the first experiments of he integrated approach '005' against 3 different optimizaiton of the design only under different pumping patterns (the third '008' uses a pattern that is the median pattern extracted from integrated approach).

In [ ]:
# exps2plot = ['nsga2__anytown_mixed_f1__exp02', 'nsga2__anytown_rehab_f1__exp04', 'nsga2__anytown_rehab_f1__fullpower', 'nsga2__anytown_rehab_f1__median']
exps2plot = ['005', '004', '007', '008']
colors = ['#29378A', '#808080', '#74BDA7','#02B9EA']
markers = ['circle', 'circle', 'square', 'diamond']
names = ['Integrated', 'Pure design - wrong - Siew et al.', 'Pure design - Full power', 'Pure design - Median pattern']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

There is something wrong, we checked and I forgot to track the intermediate steps added by EPANET. I was aware oof this but I forgot to activate this feature correctly. This is not ok but it is the default behaviour with wntr...
Energy is correct if extracted from the object,but it is not if computed starting from the values at the repèorting step (this is because they don't last for the rpeorting time step but for a shorter hydraulic timestep that get forgot).
With EPYT this does not hapen only if u use getComputedHydraulicTimeSeries.

Basically the EA was exploiting this bug finding solutions that were working at the reporting time step but failinig in intermediate ones which were not reported. making up energy and water.

## R2: v24.06.00 | Calculate Ir with all instants
Strating from this version, we fixed the problem highlihted tracking all the time steps. now we have the correct reliability.

We compare the pareto front of the experiments. Also, for the solutions in the pareto front we re-evaluate them in the other one to see what happens when the approach change. We plot the variation in the objectives.

In [ ]:
exps2plot = ['005', '011', '009', '010']
colors = ['#29378A', '#29378A', '#808080', '#808080']
markers = ['circle', 'square', 'circle', 'square']
names = ['Integrated - v24.4.0', 'Integrated - v24.6.0', 'Pure design - v24.4.0', 'Pure design - v24.6.0']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)    

versions2test = ['v24.01.00', 'v24.06.00']
plot_version_mismatch()

In the design approach doesn't change too much (going back or forward doesn't change the result that much) meaning that the approximation is good. 
However, we can't say the same about the integrated approach where solutions tend to decrease in reliability of be all together unfeasible when upgrading the version. 

## R3: Hydraulic time step dependency
We looked at the hydraulic timeseries and we saw that the Tanks were being filled and emptyed completely in less then a hydraulic timestep.
EPANET adds a simulation time step when the filling or emptying of the event would occour.
However, even if an intermediatetime step fails (water is not delivered because pressure is too low or equations can not be solved), its contribution to the reliability index objective function for the remaining of the hydrualic time step was irrelevant compated to the much higher positive contirbution to the reliability index of the few time steps that were actually working.

This could happen only the integrtaed approahc because the EA algorithm was able to turn on and off the pumps one step after the other, However, even if we constraint this behaviour we can expect that it could start doing 2 hours on, then 1 hours off and get to the same behaviour. so the whole approach is trying to do as much cycles as possible to the pumps. 

Including a constraint to avoid this behaviour would be almost impossible because a good model would need a way to translate the number of switch to an economic penalty (its clear that swithcing on and off the pumps continuously deteriorieats them and will lead to early replacement, but how much earlier?). Any choice would be just an approximation. 

So, we looked at how to avoid this in other ways. since the tank fills or emptyes completely in one hydraulic timestep the resulting simulation would be unrealistic. The tank is a dynamic element so its filling/emptying curve would follow an exponential trajectory (assuming is linear in first approximation).
The equation of a level tank is a non-linear differnetial equaiton.
I computed the time constant of the tanks by linearizing the system at the worst condition (when it takes the least amount of time to empty, ie when it is full and the head at the node where it is at the minimum required), finding that, in this case, it would be between 15 and 30 minutes. 

Than, we tried to repeat the experiment with shorter hydraulic timesteps

In [ ]:
exps2plot = ['011', '013', '015',
             '010', '012', '014']
colors = ['#29378A', '#29378A', '#29378A',
          '#808080', '#808080', '#808080']
markers = ['circle', 'square', 'diamond',
           'circle', 'square', 'diamond']
names = ['Integrated - 60min', 'Integrated - 30min', 'Integrated - 15min',
        'Pure design - 60min', 'Pure design - 30min', 'Pure design - 15min']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

Clearly, the hydraulic time step plays a huge role in the solutions feasibility. While with the design only approach this was not too significative (because the pumping action was more or less constant duing the day and between the hydraulic time steps) so it oculd not really play with the dynamics of the network

with the integrated it plays a huge role. because it prefers network with a lot of power surplus at the beginning of the hydraulic tiestpe and neglects what happens for the rest of the hydraulic time step as long as the average is good enough. 

## Exisisting pipes formulation 1 (Farmani) vs formulation 2 (mine & Mark)

1. fexp1: [0: do nothing, 1 clean, 2 install] x [0->9: pipe rehabilitation alternative (active only if previous set to 2)]

2. fexp2: [0: do nothing, 1 clean, 2->11 pipe rehabilitation_alternative]

In [ ]:
exps2plot = ['015', '017', '014', '016']
colors = ['#29378A', '#29378A', '#808080', '#808080']
markers = ['circle', 'square', 'circle', 'square']
names = ['Integrated - Existing Pipe F1', 'Integrated - Existing Pipe F2', 'Pure design - Existing Pipe F1', 'Pure design - Existing Pipe F2']
legendgroup_titles = ['Integrated', 'Integrated', 'Pure design', 'Pure design']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

# Plot the HV to see if the f2 works better than f1. One HV value per each generation. Do it for each island
fig = go.Figure()
hv_refpoint = [0,0]
linestyles = ['solid', 'dashdot', 'solid', 'dashdot']
for e, expname in enumerate(exps2plot):
    genz = experiments[expname].generations
    fvs = experiments[expname].fitness_vectors
    # I will draw one line for each island
    for i, isl in enumerate(list(experiments[expname].data['archipelago']['islands'].keys())):
        n_reps = genz[isl].shape[0]
        hv = np.zeros(genz[isl].shape[0])
        for g, gen in enumerate(genz[isl]):
            datapoint = fvs.loc[isl].loc[gen].to_numpy()
            datapoint = datapoint[np.logical_and(datapoint[:,0]>0, datapoint[:,1]<0, datapoint[:,0] <= 20e6), :]
            if (datapoint.shape[0] > 0):
                datapoint[:,0] = -datapoint[:,0]/20e6
                hv[g] = pg.hypervolume(datapoint).compute(hv_refpoint)

        fig.add_trace(go.Scatter(x=genz[isl], y=hv, mode='lines',
                                marker=dict(size=msize, symbol=markers[e], color=colors[e]),
                                line=dict(width=2, dash=linestyles[e]),
                                showlegend=True, name=isl,
                                legendgroup=expname,
                                legendgrouptitle_text=names[e],
                                customdata=np.array(np.arange(genz[isl].shape[0]), dtype=str),
                                hovertemplate='Generation: %{x} <br> HV: %{y:.2f} <extra>%{customdata}</extra>'
                                ))

#fix_layout(fig, 'Hypervolume of the different solutions')
fig.update_layout(legend=dict(groupclick="toggleitem"))
fig.show()

# Plot the re-evaluation on the different formulations
formulations2test = [1, 2]
plot_formulation_mismatch()

So we have a double benefit with this formulation:
1. in the integrated, more seeds converge.
2. in both, we don't really converge sooner, but we started from more feasible solutions...

## v24.11.0 aka BUG: check for invalid solutions in DDA

I was checking Head > 0 while I should have been checking Head > Elevation.
This bug should have no or little impact as the soultions with Head > 0 but < Elevetaion should be few.

In [ ]:
# Same as before. We need to check v24.10.0 vs 24.11.0 for the mixed and rehab solutions
exps2plot = ['017', '019', '016', '018']
colors = ['#29378A', '#29378A', '#808080', '#808080']
markers = ['circle', 'square', 'circle', 'square']
names = ['Integrated - v24.10.0', 'Integrated - v24.11.0', 'Pure design - v24.10.0', 'Pure design - v24.11.0']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

versions2test = ['v24.10.00', 'v24.11.00']
plot_version_mismatch()

Nothing changed, as I was expecting. 

## v24.12.0 aka inverted riser
Point: this should not be possible. A parameter should not hcange the mathematial solution. This hidners reproducibility. It doesn't invalide anything.
It would be interesting to quantify it for a generic network. 

Solution: A different heuristic to set the intial flows (it may even help with convergence).


In [ ]:
# Same as before. We need to check v24.11.0 vs 24.12.0 for the mixed and rehab solutions
exps2plot = ['019', '025', '018', '024']
colors = ['#29378A', '#29378A', '#808080', '#808080']
markers = ['circle', 'square', 'circle', 'square']
names = ['Integrated - v24.10.0', 'Integrated - v24.12.0', 'Pure design - v24.10.0', 'Pure design - v24.12.0']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

versions2test = ['v24.11.00', 'v24.12.00']
plot_version_mismatch()

## V25.02.00 aka New EPANET version (after bug)

Point: we need better issue tracking. This point hinders reproducibility. 

Solutio: add patch version to EPANET 2.3.1 etc 

In [ ]:
# Same as before. We need to check v24.12.0 vs 25.02.0 for the mixed and rehab solutions
exps2plot = ['025', '031', '024', '030']
colors = ['#29378A', '#29378A', '#808080', '#808080']
markers = ['circle', 'square', 'circle', 'square']
names = ['Integrated - v24.12.0', 'Integrated - v25.02.0', 'Pure design - v24.12.0', 'Pure design - v25.02.0']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

versions2test = ['v24.12.00', 'v25.02.00']
plot_version_mismatch()

## Formulation 2 vs Formulation 3 (new objective function)

Point: the results of EPANET should be a hard constraint to consider. Implemeting the failure as a "soft" constraint doesn't work.

1. fof1: failure is accounting putting reliability at 0 at that step.
2. fof2: failure is checked before everything and it is reuqired (hard constrained)

The prolem with the first one is that the EA, still manages to find solutions hat maybe are unfeasible for a little bit instead of going for fully feasible solutions that work worst.

In [ ]:
# Same as before. We need to check v25.02.0 vs 25.03.0 for the mixed and rehab solutions
exps2plot = ['030', '032', '031', '033']
colors = ['#808080', '#808080', '#29378A', '#29378A']
markers = ['circle', 'square', 'circle', 'square']
names = ['Pure design :: f2', 'Pure design :: f3', 'Integrated :: f2', 'Integrated :: f3']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

formulations2test = [2, 3]
plot_formulation_mismatch()

In [ ]:
# Same as before. We need to check v25.02.0 vs 25.03.0 for the mixed and rehab solutions
exps2plot = ['032', '034', '033', '035']
colors = ['#808080', '#808080', '#29378A', '#29378A']
markers = ['circle', 'square', 'circle', 'square']
names = ['Pure design :: f3', 'Pure design :: f4', 'Integrated :: f3', 'Integrated :: f4']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

formulations2test = [3, 4]
plot_formulation_mismatch()

In [ ]:
# Just test that the same problem evaluated in another formualtion gives the same results... 
# I get errors in the nano magnitude because of how floating point operations are implemented.
# Specifically, the min volume turns out to be different..
exps2plot = ['036', '036', '037', '037']
colors = ['#808080', '#808080', '#29378A', '#29378A']
markers = ['circle', 'square', 'circle', 'square']
names = ['Pure design :: f4', 'Pure design :: f4', 'Integrated :: f4', 'Integrated :: f4']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

formulations2test = [4, 5]
plot_formulation_mismatch()

In [ ]:
exps2plot = ['037', '039', '036', '038']
colors = ['#29378A', '#29378A', '#808080', '#808080']
markers = ['circle', 'square', 'circle', 'square']
names = ['Integrated::F4', 'Integrated::F5', 'Pure design::F4', 'Pure design::F5']
legendgroup_titles = ['Integrated', 'Integrated', 'Pure design', 'Pure design']
plot_pareto_fronts(experiments, exps2plot, colors, markers, names)

# Plot the HV to see if the f2 works better than f1. One HV value per each generation. Do it for each island
fig = go.Figure()
hv_refpoint = [0,0]
linestyles = ['solid', 'dashdot', 'solid', 'dashdot']
for e, expname in enumerate(exps2plot):
    genz = experiments[expname].generations
    fvs = experiments[expname].fitness_vectors
    # I will draw one line for each island
    for i, isl in enumerate(list(experiments[expname].data['archipelago']['islands'].keys())):
        n_reps = genz[isl].shape[0]
        hv = np.zeros(genz[isl].shape[0])
        for g, gen in enumerate(genz[isl]):
            datapoint = fvs.loc[isl].loc[gen].to_numpy()
            datapoint = datapoint[np.logical_and(datapoint[:,0]>0, datapoint[:,1]<0, datapoint[:,0] <= 20e6), :]
            if (datapoint.shape[0] > 0):
                datapoint[:,0] = -datapoint[:,0]/20e6
                hv[g] = pg.hypervolume(datapoint).compute(hv_refpoint)

        fig.add_trace(go.Scatter(x=genz[isl], y=hv, mode='lines',
                                marker=dict(size=msize, symbol=markers[e], color=colors[e]),
                                line=dict(width=2, dash=linestyles[e]),
                                showlegend=True, name=isl,
                                legendgroup=expname,
                                legendgrouptitle_text=names[e],
                                customdata=np.array(np.arange(genz[isl].shape[0]), dtype=str),
                                hovertemplate='Generation: %{x} <br> HV: %{y:.2f} <extra>%{customdata}</extra>'
                                ))

#fix_layout(fig, 'Hypervolume of the different solutions')
fig.update_layout(legend=dict(groupclick="toggleitem"))
fig.show()
